In [750]:
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from config import api_key
%matplotlib inline

In [751]:
quandl.ApiConfig.api_key = api_key
#Variance Risk Premium: https://sites.google.com/site/haozhouspersonalhomepage
#EOM Nominal S&P, Nominal Dividends / Nominal Risk Free Rate: http://www.hec.unil.ch/agoyal/
#Consumer Sentiment: http://www.sca.isr.umich.edu/

In [752]:
ticker = 'SPY'
transform = 'normalize'
start_date = '1989-01-01'
end_date = '2018-01-01'

In [753]:
index_data = quandl.get('EOD/' + ticker, start_date=start_date, end_date=end_date, column_index=4, collapse='monthly')

In [754]:
risk_free_rate = quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date, collapse='monthly')

In [755]:
fed_funds_rate = quandl.get('FRED/FEDFUNDS', start_date=start_date, end_date=end_date, collapse='monthly')

In [756]:
index_data = index_data.reset_index()
risk_free_rate = risk_free_rate.reset_index()
fed_funds_rate = fed_funds_rate.reset_index()

In [757]:
#Calculate Monthly FFR
fed_funds_rate_list = []

for i in range(len(fed_funds_rate)):
    x = (fed_funds_rate.iloc[i]['Value']/12)/100
    fed_funds_rate_list.append(x)

In [758]:
"""
Monthly >>>
Index: S&P 500 Index Data
D12: Dividends are 12-month moving sums of dividends paid on the S&P 500 index (Goyal, Welch) 
     -- Confirmed with Y Charts
Rfree: Nominal Risk Free Rate
"""
goyal = pd.read_excel('PredictorData2017.xlsx')
df = pd.DataFrame(goyal)
df.head()

'\nMonthly >>>\nIndex: S&P 500 Index Data\nD12: Dividends are 12-month moving sums of dividends paid on the S&P 500 index (Goyal, Welch) \n     -- Confirmed with Y Charts\nRfree: Nominal Risk Free Rate\n'

,yyyymm,Index,D12,E12,b/m,tbl,AAA,BAA,lty,ntis,Rfree,infl,ltr,corpr,svar,csp,CRSP_SPvw,CRSP_SPvwx
0,187101,4.44,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004955,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,187102,4.50,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004514,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,187103,4.61,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004243,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,187104,4.74,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.004632,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,187105,4.86,0.26,0.4,NaN,NaN,NaN,NaN,NaN,NaN,0.003691,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [759]:
#New DF with cleaned up Columns
df_cols = ['yyyymm', 'Index', 'D12', 'Rfree']
df = df[df_cols]

In [760]:
#Convert yyyymm to Datetime then to Period
yyyymm = pd.to_datetime(df['yyyymm'], format='%Y%m').dt.strftime('%Y-%m')
periods = pd.to_datetime(yyyymm, format='%Y-%m')
df['yyyymm'] = periods.dt.to_period('M')

In [761]:
df.head()
df.tail()
df.info()

,yyyymm,Index,D12,Rfree
0,1871-01,4.44,0.26,0.004955
1,1871-02,4.50,0.26,0.004514
2,1871-03,4.61,0.26,0.004243
3,1871-04,4.74,0.26,0.004632
4,1871-05,4.86,0.26,0.003691


,yyyymm,Index,D12,Rfree
1759,2017-08,2471.649902,47.855075,0.000842
1760,2017-09,2519.360107,48.173103,0.000858
1761,2017-10,2575.260010,48.426075,0.000892
1762,2017-11,2584.840088,48.679047,0.001025
1763,2017-12,2673.610107,48.932019,0.001100


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1764 entries, 0 to 1763
Data columns (total 4 columns):
yyyymm    1764 non-null period[M]
Index     1764 non-null float64
D12       1764 non-null float64
Rfree     1764 non-null float64
dtypes: float64(3), period[M](1)
memory usage: 55.2 KB


In [762]:
#Drop rows prior to 1990-01
df = df[df['yyyymm'] >= '1990-01']

In [763]:
df.head()
df.tail()
df.shape

,yyyymm,Index,D12,Rfree
1428,1990-01,329.08,11.142667,0.006367
1429,1990-02,331.89,11.230333,0.006450
1430,1990-03,339.94,11.318000,0.006583
1431,1990-04,330.80,11.433000,0.006475
1432,1990-05,361.23,11.548000,0.006450


,yyyymm,Index,D12,Rfree
1759,2017-08,2471.649902,47.855075,0.000842
1760,2017-09,2519.360107,48.173103,0.000858
1761,2017-10,2575.260010,48.426075,0.000892
1762,2017-11,2584.840088,48.679047,0.001025
1763,2017-12,2673.610107,48.932019,0.001100


(336, 4)

In [764]:
"""
VRP: Variance Risk Premium = IV - RV
IV: Implied Variance = VIX-Squared/12
RV: Realized Variance = Sum of Squared 5-minute log returns of the S&P 500 Index
"""
vrp = pd.read_fwf('VRPtable.txt')
df_vrp = pd.DataFrame(vrp)
df_vrp.tail(10)
df_vrp.shape

'\nVRP: Variance Risk Premium = IV - RV\nIV: Implied Variance = VIX-Squared/12\nRV: Realized Variance = Sum of Squared 5-minute log returns of the S&P 500 Index\n'

,Year,Month,VRP,EVRP,IV,RV,ERV
338,2018,3,8.6653,5.7620,33.2334,24.5681,27.4714
339,2018,4,-1.9751,-8.2494,21.1471,23.1221,29.3964
340,2018,5,9.4048,3.2582,19.8404,10.4356,16.5822
341,2018,6,13.2745,5.0526,21.5740,8.2995,16.5214
342,2018,7,8.1409,-4.8897,13.7174,5.5765,18.6072
343,2018,8,8.5030,3.3157,13.7816,5.2786,10.4659
344,2018,9,7.9528,1.6749,12.2412,4.2884,10.5663
345,2018,10,-2.2509,28.5411,37.5594,39.8103,9.0183
346,2018,11,2.7329,-5.2160,27.2104,24.4775,32.4264
347,2018,12,-4.0034,30.9039,53.8480,57.8514,22.9441


(348, 7)

In [765]:
#Drop 2018 Rows
df_vrp = df_vrp[df_vrp['Year'] < 2018]

In [766]:
df_vrp.head()
df_vrp.tail()
df_vrp.shape

,Year,Month,VRP,EVRP,IV,RV,ERV
0,1990,1,35.9054,35.9054,53.5941,17.6888,17.6888
1,1990,2,32.9177,15.4492,40.2967,7.3790,24.8475
2,1990,3,26.5978,16.0745,32.4394,5.8416,16.3649
3,1990,4,26.2753,18.6211,31.7525,5.4773,13.1314
4,1990,5,19.2201,12.3841,25.1431,5.9229,12.7589


,Year,Month,VRP,EVRP,IV,RV,ERV
331,2017,8,4.5638,5.2881,9.3457,4.7819,4.0576
332,2017,9,4.9171,2.5581,7.5367,2.6196,4.9786
333,2017,10,6.5894,5.1081,8.6360,2.0466,3.5279
334,2017,11,6.9722,6.9307,10.6032,3.6310,3.6725
335,2017,12,6.1073,5.2063,10.1568,4.0495,4.9505


(336, 7)

In [767]:
#Drop Unnecessary Columns
df_vrp = df_vrp[['VRP', 'IV', 'RV']]

In [768]:
df_vrp.head()

,VRP,IV,RV
0,35.9054,53.5941,17.6888
1,32.9177,40.2967,7.3790
2,26.5978,32.4394,5.8416
3,26.2753,31.7525,5.4773
4,19.2201,25.1431,5.9229


In [769]:
df = pd.concat([df.reset_index(drop=True), df_vrp.reset_index(drop=True)], axis=1)
df.head()

,yyyymm,Index,D12,Rfree,VRP,IV,RV
0,1990-01,329.08,11.142667,0.006367,35.9054,53.5941,17.6888
1,1990-02,331.89,11.230333,0.006450,32.9177,40.2967,7.3790
2,1990-03,339.94,11.318000,0.006583,26.5978,32.4394,5.8416
3,1990-04,330.80,11.433000,0.006475,26.2753,31.7525,5.4773
4,1990-05,361.23,11.548000,0.006450,19.2201,25.1431,5.9229


In [770]:
#Rename yyyymm Column to EOM
df = df.rename(columns={'yyyymm': 'EOM'})

In [771]:
#New DF from Index / Div / Rfree Data -> Need to Add 1989 Data.
df_1989 = pd.DataFrame(goyal)
df_1989_cols = ['yyyymm', 'Index', 'D12', 'Rfree']
df_1989 = df_1989[df_1989_cols]
yyyymm_1989 = pd.to_datetime(df_1989['yyyymm'], format='%Y%m').dt.strftime('%Y-%m')
periods_1989 = pd.to_datetime(yyyymm_1989, format='%Y-%m')
df_1989['yyyymm'] = periods_1989.dt.to_period('M')
df_1989 = df_1989[(df_1989['yyyymm'] >= '1989-01') & (df_1989['yyyymm'] < '1990-01')]
df_1989

,yyyymm,Index,D12,Rfree
1416,1989-01,297.47,9.837,0.006892
1417,1989-02,288.86,9.924,0.007108
1418,1989-03,294.87,10.011,0.007350
1419,1989-04,309.64,10.130,0.007208
1420,1989-05,320.52,10.249,0.007025
1421,1989-06,317.98,10.368,0.006792
1422,1989-07,346.08,10.490,0.006567
1423,1989-08,351.45,10.612,0.006583
1424,1989-09,349.15,10.734,0.006458
1425,1989-10,340.36,10.841,0.006367


In [772]:
df_1989 = df_1989.rename(columns={'yyyymm': 'EOM'})
df_1989

,EOM,Index,D12,Rfree
1416,1989-01,297.47,9.837,0.006892
1417,1989-02,288.86,9.924,0.007108
1418,1989-03,294.87,10.011,0.007350
1419,1989-04,309.64,10.130,0.007208
1420,1989-05,320.52,10.249,0.007025
1421,1989-06,317.98,10.368,0.006792
1422,1989-07,346.08,10.490,0.006567
1423,1989-08,351.45,10.612,0.006583
1424,1989-09,349.15,10.734,0.006458
1425,1989-10,340.36,10.841,0.006367


In [773]:
df = pd.concat([df_1989, df], axis=0, ignore_index=True, sort=False)

In [774]:
"""
Get monthly average Federal Funds Rate:
https://fred.stlouisfed.org/series/FEDFUNDS
***Is this divided by 12?***
"""
fed_funds_file = pd.read_csv('FEDFUNDS (1).csv')
fed_funds = pd.DataFrame(fed_funds_file)
fed_funds.head()

'\nGet monthly average Federal Funds Rate:\nhttps://fred.stlouisfed.org/series/FEDFUNDS\n***Is this divided by 12?***\n'

,DATE,FEDFUNDS
0,1989-01-01,9.12
1,1989-02-01,9.36
2,1989-03-01,9.85
3,1989-04-01,9.84
4,1989-05-01,9.81


In [775]:
fed_funds['FEDFUNDS'] = (fed_funds['FEDFUNDS'])

In [776]:
fed_funds = fed_funds['FEDFUNDS']
df = pd.concat([df.reset_index(drop=True), fed_funds.reset_index(drop=True)], axis=1)
df = df.rename(columns={'FEDFUNDS': 'fed_funds'})

In [777]:
df.head()

,EOM,Index,D12,Rfree,VRP,IV,RV,fed_funds
0,1989-01,297.47,9.837,0.006892,NaN,NaN,NaN,9.12
1,1989-02,288.86,9.924,0.007108,NaN,NaN,NaN,9.36
2,1989-03,294.87,10.011,0.007350,NaN,NaN,NaN,9.85
3,1989-04,309.64,10.130,0.007208,NaN,NaN,NaN,9.84
4,1989-05,320.52,10.249,0.007025,NaN,NaN,NaN,9.81


In [778]:
sentiment_file = 'tbmics.csv'
sentiment = pd.read_csv(sentiment_file)
sentiment.head(10)

,Month,YYYY,ICS_ALL
0,November,1952,86.2
1,February,1953,90.7
2,August,1953,80.8
3,November,1953,80.7
4,February,1954,82.0
5,May,1954,82.9
6,November,1954,87.0
7,February,1955,95.9
8,May,1955,99.1
9,November,1955,99.7


In [779]:
sentiment = sentiment[(sentiment['YYYY'] >= 1989) & (sentiment['YYYY'] < 2018)]
sentiment = sentiment['ICS_ALL']

In [780]:
df = pd.concat([df.reset_index(drop=True), sentiment.reset_index(drop=True)], axis=1)

In [781]:
"""
FORMULAS:

Two Gross Nominal Return Formulas. Paper defines Gross Nominal Return as:
(P_t + (D_t / 12)) / P_t-1) --> Not Standard formula, results in 1.011383, is this intentional?
"""
#Number of Periods in Variance Risk Premium
vrp_rolling_periods = 3

#Calculate Gross Nominal Return
gross_nominal_return_1 = ((df['Index'] + df['D12'] / 12) - df['Index'].shift(1))/ df['Index'].shift(1)
#gross_nominal_return_2 = ((df['Index'] + df['D12'] / 12))/ df['Index'].shift(1) - 1

#Calculate Price Dividend Ratio
price_dividend_ratio = (df['Index'] / df['D12'])

#Calculate Fed Funds Rate Delta
fed_funds_delta_formula = df['fed_funds'] - df['fed_funds'].shift(12)

#Calculate Variance Risk Premium Rolling
rolling_vrp_formula = df['VRP'].rolling(vrp_rolling_periods).mean()

#Calculate Sentiment Delta
sent_delta_formula = df['ICS_ALL'] - df['ICS_ALL'].shift(12);

In [782]:
df['g_nominal_return'] = gross_nominal_return_1
df['price_dividend'] = price_dividend_ratio
df['fed_funds_delta'] = fed_funds_delta_formula
df['rolling_VRP'] = rolling_vrp_formula
df['sent_delta'] = sent_delta_formula

ersf_delta_formula = df['g_nominal_return'] - df['g_nominal_return'].shift(1)
df['ersf_delta'] = ersf_delta_formula

df['sent_v_ersf'] = df['sent_delta'] * df['ersf_delta']

In [783]:
df['ersf_t1'] = (df['g_nominal_return'].shift(-1) - df['Rfree'].shift(-1)) * 100

In [787]:
df.head()

,EOM,Index,D12,Rfree,VRP,IV,RV,fed_funds,ICS_ALL,g_nominal_return,price_dividend,fed_funds_delta,rolling_VRP,sent_delta,ersf_delta,sent_v_ersf,ersf_t1
0,1989-01,297.47,9.837,0.006892,NaN,NaN,NaN,9.12,97.9,NaN,30.239911,NaN,NaN,NaN,NaN,NaN,-3.327232
1,1989-02,288.86,9.924,0.007108,NaN,NaN,NaN,9.36,95.4,-0.026164,29.107215,NaN,NaN,NaN,NaN,NaN,1.634400
2,1989-03,294.87,10.011,0.007350,NaN,NaN,NaN,9.85,94.3,0.023694,29.454600,NaN,NaN,NaN,0.049858,NaN,4.574438
3,1989-04,309.64,10.130,0.007208,NaN,NaN,NaN,9.84,91.5,0.052953,30.566634,NaN,NaN,NaN,0.029259,NaN,3.087089
4,1989-05,320.52,10.249,0.007025,NaN,NaN,NaN,9.81,90.7,0.037896,31.273295,NaN,NaN,NaN,-0.015057,NaN,-1.202067


In [785]:
#Create Filter for 1990 and Later
filt_1990 = df['EOM'] >= '1990-03'
dfx = df[filt_1990]

In [786]:
dfx.describe()

,Index,D12,Rfree,VRP,IV,RV,fed_funds,ICS_ALL,g_nominal_return,price_dividend,fed_funds_delta,rolling_VRP,sent_delta,ersf_delta,sent_v_ersf,ersf_t1
count,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,334.000000,333.000000
mean,1163.313893,22.396469,0.002270,17.307943,35.987940,16.003604,2.937904,86.840719,0.008851,52.045820,-0.289132,17.390332,0.176347,0.000073,-0.035804,0.653912
std,543.692853,10.218236,0.001883,13.929337,33.605379,15.969630,2.408457,12.484535,0.041014,14.204175,1.405820,11.135081,10.053697,0.056828,0.573707,4.105890
min,304.000000,11.318000,0.000008,-9.724000,7.536700,1.210800,0.070000,55.300000,-0.167374,25.507636,-4.580000,3.052700,-30.000000,-0.135668,-2.934586,-16.793245
25%,764.380000,15.019000,0.000179,8.354675,15.606600,5.812950,0.225000,77.600000,-0.015696,44.239881,-0.757500,9.454383,-4.250000,-0.036721,-0.182315,-1.678295
50%,1158.590000,17.704500,0.002408,13.408600,25.230000,10.136800,3.000000,88.950000,0.012539,51.087280,-0.030000,13.623633,1.100000,-0.000367,-0.002318,1.016117
75%,1415.175000,27.594417,0.004115,23.482400,45.923100,19.242050,5.250000,94.925000,0.035407,58.380221,0.297500,23.211900,6.500000,0.033130,0.132628,3.335128
max,2673.610107,48.932019,0.006583,80.610800,298.901000,96.730100,8.290000,112.000000,0.114298,92.170533,2.670000,59.880367,22.800000,0.208402,2.793085,11.090639
